In [242]:
from osmapi import OsmApi
from geojson import LineString, GeometryCollection, Point
import folium
from collections import deque

In [243]:
MyApi = OsmApi()

def Map(	self, min_lon, min_lat, max_lon, max_lat)
Download data in bounding box. Returns list of dict { type: node|way|relation, data: {} }.

In [374]:
highway_tags = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                            'service', 'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link',
                            'living_street', 'pedestrian', 'road']
#Raw Data
map_data = MyApi.Map(22.216864,60.416923,22.286005,60.451442)
way_data = [row for row in map_data if row['type'] == 'way']
node_data = [row for row in map_data if row['type'] == 'node']
highway_raw = [row for row in way_data if 'highway' in row['data']['tag'].keys()]
highway_data = [row for row in highway_raw for tag in highway_tags if tag in row['data']['tag']['highway']]
link_data = [row for row in way_data if 'highway' in row['data']['tag'].keys() and 'link' in row['data']['tag']['highway']]

In [375]:
#Helper functions
def node2loc(node_id):
    for node in node_data:
        if node_id == node['data']['id']:
            return (node['data']['lon'], node['data']['lat'])

In [376]:
#All Roads to GeoJson
roads = []
for highway in highway_data:
    road = []
    nodes = highway['data']['nd'] #List of nodes in highway
    for node in nodes:
        road.append(node2loc(node))
    roads.append(LineString(road))
geo_json = GeometryCollection(roads)

In [377]:
m = folium.Map(location=[ 60.4518, 22.2666], zoom_start=15)
folium.GeoJson(geo_json,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)

In [378]:
m

In [379]:
#Road turn
turns = set()
for highway_1 in highway_data:
    nodes_1 = highway_1['data']['nd']
    for node_1 in nodes_1:
        for highway_2 in highway_data: 
            if highway_2 != highway_1:
                nodes_2 = highway_2['data']['nd']
                for node_2 in nodes_2:
                    if node_2 == node_1:
                        turns.add(node2loc(node_1))

turn_geojson = GeometryCollection([Point(turn) for turn in list(turns)])

In [380]:
m = folium.Map(location=[60.4518, 22.2666], zoom_start=15)
folium.GeoJson(geo_json,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)
folium.GeoJson(turn_geojson,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)

In [381]:
m

In [641]:
from osmapi import OsmApi
from geojson import LineString, GeometryCollection, Point
from math import radians, sin, cos, sqrt, asin
import heapq

class Aplanner(object):
    """ Process data in a bounding box to produce data structures needed for
        A* search
    """
    def __init__(self, lon1,  lat1, lon2, lat2):
        self.MyApi = OsmApi()
        
        #List of allowed road types
        self.highway_tags = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                            'service', 'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link',
                            'living_street', 'pedestrian', 'road']
        
        #Extract basic data units
        self.bounding_box = [lon1,  lat1, lon2, lat2]
        self.map_data = self.MyApi.Map(lon1,  lat1, lon2, lat2)
        self.way_data = [row for row in self.map_data if row['type'] == 'way']
        self.node_data = [row for row in self.map_data if row['type'] == 'node']
        self.highway_data = [row for row in self.way_data if 'highway' in row['data']['tag'].keys()]
        #self.highway_data = [row for row in self.highway_raw for tag in self.highway_tags if tag in row['data']['tag']['highway']]
        
        #Node_id to lon lat
        self.node_ids = [node['data']['id'] for node in self.node_data]
        #self.node_id_loc = {node_id:self.node2loc(node_id) for node_id in self.node_ids}
        
        #Dictionary of road_id and list of node ID's
        self.roads = {highway['data']['id']:highway['data']['nd'] for highway in self.highway_data}
        
        #Dictionary of node_id and list of road_ID's
        self.node_roads = self.node_roads_dict()
        
        #List of unique nodes at road changes or turns
        self.turns = self.turn_extractor(self.roads)
        
        #Road_id and their turns
        self.road_turns = self.road_turns(self.roads)
        
        self.neighbours = self.neigbour_nodes()
        
        
          
    #Helper functions
    def node2loc(self, node_id):
        for node in self.node_data:
            if node_id == node['data']['id']:
                return (node['data']['lon'], node['data']['lat'])
    
    
    def turn_extractor(self, roads):
        turns = set()
        for road_1 in self.roads.keys():
            nodes_1 = self.roads[road_1]
            for node_1 in nodes_1:
                for road_2 in self.roads.keys(): 
                    if road_2 != road_1:
                        nodes_2 = self.roads[road_2]
                        for node_2 in nodes_2:
                            if node_2 == node_1:
                                turns.add(node_1)
        return list(turns)
    
    
    def road_turns(self, roads):
        road_turns = {}
        for key in self.roads.keys():
            temp_turns = []
            for node in roads[key]:
                if node in self.turns:
                    temp_turns.append(node)
            road_turns[key] = temp_turns
        return road_turns
    
    
    def node2road(self, node_id):
        return self.node_roads[node_id] #List of roads node is in
    
    
    def node_roads_dict(self):
        node_roads = {}
        for road in list(self.roads.keys()):
            for node in self.roads[road]:
                if node not in list(node_roads.keys()):
                    node_roads[node] = [road]
                else:
                    list_ = node_roads[node]
                    list_.append(road)
                    node_roads[node] = list_
        return node_roads
    
    
    def neigbour_nodes(self):
        neighbours_dict = {}
        for road in self.roads:
            for index, node in enumerate(self.roads[road]):
                forward_node = index + 1
                backward_node = index - 1
                try:
                    forward = self.roads[road][forward_node]
                    if node not in neighbours_dict:
                        neighbours_dict[node] = [forward]
                    else:
                        list_ = neighbours_dict[node]
                        list_.append(forward)
                        neighbours_dict[node] = list_
                except:
                    pass
                if backward_node >= 0:
                    try:
                        backward = self.roads[road][backward_node]
                        if node not in neighbours_dict:
                            neighbours_dict[node] = [backward]
                        else:
                            list_ = neighbours_dict[node]
                            list_.append(backward)
                            neighbours_dict[node] = list_
                    except:
                        pass 
        return neighbours_dict
    
    def neighbour_list(self, node_id):
        return self.neighbours[node_id]
    
    def node_path_geojson(self, node_list):
        roads = []
        for node in node_list:
            roads.append(self.node2loc(node))
        return LineString(roads)
    
    def haversine(self, lat1, lon1, lat2, lon2):
        R = 6372.8 # Earth radius in kilometers
        dLat = radians(lat2 - lat1)
        dLon = radians(lon2 - lon1)
        lat1 = radians(lat1)
        lat2 = radians(lat2)
 
        a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
        c = 2*asin(sqrt(a))
        return R * c
    
    def node_distance(self, node1, node2):
        lon1, lat1 = self.node2loc(node1)
        lon2, lat2 = self.node2loc(node2)
        distance = self.haversine(lat1, lon1, lat2, lon2)
        return distance
    
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]
        

        
        #turn_geojson = GeometryCollection([Point(turn) for turn in list(turns)])

In [629]:
from time import time

def breadth_first_search(graph, start, goal):
    frontier = deque([])
    frontier.append(start)
    came_from = {}
    came_from[start] = None
    
    while len(frontier) > 0:
        current = frontier.popleft()
        if current == goal:
            print("Solution Found")
            break
        children = graph.neighbour_list(current)
        for child in children:
            if child not in came_from:
                frontier.append(child)
                came_from[child] = current
    return came_from

def uniform_cost_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        #Check if the goal has been expanded
        if goal == current:
            print("Solution Found")
            break
        
        for child in graph.neighbour_list(current):
            child_cost = cost_so_far[current] + graph.node_distance(current, child)
            if child not in came_from or child_cost < cost_so_far[child]:
                priority = child_cost
                frontier.put(child, priority)
                came_from[child] = current
                cost_so_far[child] = child_cost
        if frontier.empty():
            print("No Solution")
                
    return came_from

def path_construction(start, node, search_results):
    if node == start:
        return [node]
    else:
        list_ = [node]
        list_.extend(path_construction(start, search_results[node], search_results))
        return list_

In [642]:
#Load data
graph = Aplanner(22.216864,60.416923,22.286005,60.451442)

In [646]:
#Define Problem
start =  21646152
goal = 21646230



#start = 21646234
#goal = 21646152
#goal = 21646234

In [649]:
search_results = breadth_first_search(graph, start, goal)
path_1 = path_construction(start, goal, search_results)
search_results = uniform_cost_search(graph, start, goal)
path_2 = path_construction(start, goal, search_results)

Solution Found
Solution Found


In [650]:
m = folium.Map(location=[ 60.4218, 22.2666], zoom_start=14)
folium.GeoJson(graph.node_path_geojson(path_2),
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3,
        'opacity': 1,
        'fillColor' : 'black'
        }).add_to(m)
m

TypeError: neigbour_nodes() takes 1 positional argument but 2 were given

In [626]:
graph.neighbour_list(1101337726)

[243674925, 21646150, 1101338009, 1101337902]

In [627]:
graph.neighbour_list(21646150)

[1101337726, 21646152, 553309834, 1851754323, 1252511543, 1101337895]

In [556]:
graph.roads[42684571]

[21646150, 1101337726, 243674925, 533774011, 32919954, 30390838, 21646152]

In [573]:
graph.node2road(21646230)

[22649715, 43054217, 43054312]

In [557]:
graph.roads[22649715]

[21646230,
 1257246247,
 21646231,
 21646232,
 1397057780,
 21646233,
 162312303,
 1397057781,
 1404046778,
 21646234,
 21646235,
 272439030,
 21646237,
 4172653517,
 1030550796,
 1030550799,
 1030550820,
 1030550720,
 1030550814,
 21646241,
 28892366,
 1101337760,
 1101337895,
 21646150]

In [567]:
path_1

[21646230,
 28892362,
 1404046031,
 28892360,
 28892358,
 471071583,
 4172653495,
 1257246282,
 21855862,
 21855861,
 1257246253,
 21855860,
 1257246231,
 21855859,
 1257246290,
 21855858,
 28892364,
 530866341,
 3223232626,
 3223232625,
 1252511512,
 21855854,
 530866234,
 756885115,
 1252511533,
 21855855,
 4220066034,
 1101337815,
 2152981395,
 394515300,
 26936788,
 256592107,
 26936787,
 1045779600,
 26910546,
 1045780124,
 1045779941,
 2152981403,
 25370196,
 1051695751,
 1274616345,
 256830324,
 1032763273,
 26936767,
 26936769,
 26936765,
 26936764,
 663204145,
 1851754318,
 4310018990,
 3220098710,
 4309622475,
 242900856,
 242900857,
 242900858,
 242900859,
 242900860,
 388314934,
 553309827,
 533774011,
 32919954,
 30390838,
 21646152]

In [512]:
graph.node_path_geojson([21855869,
 554190049,
 1252511515,
 30070345,
 1252511439,
 1274616355,
 1252511546,
 1252511468,
 21402480,
 256371984,
 597376717,
 1334702233,
 21402481,
 984624826,
 1334702231,
 283313787,
 558546949,
 215546743,
 215546699,
 1334702241,
 471079552,
 21855854,
 530866234,
 756885115,
 1252511533,
 21855855,
 1101337653,
 21646201,
 3223179595,
 2601642846,
 3223179632,
 3223179631,
 597376713,
 1101337825,
 1099808365,
 1099808361,
 1101337543,
 1101337632,
 1252511543,
 21646150])

{"coordinates": [[22.2747865, 60.4355949], [22.274602, 60.4355008], [22.2709347, 60.4338668], [22.2707958, 60.4338038], [22.2703327, 60.4335982], [22.2690885, 60.4330459], [22.2681747, 60.4326403], [22.2676026, 60.4324442], [22.267359, 60.4323771], [22.2671311, 60.432333], [22.266881, 60.432289], [22.2667089, 60.4322617], [22.2665567, 60.4322376], [22.2663599, 60.4322148], [22.265596, 60.4321903], [22.2652534, 60.4321911], [22.261958, 60.4322793], [22.2618602, 60.4322833], [22.2600574, 60.4323299], [22.2586309, 60.4323584], [22.2580002, 60.4323507], [22.2577353, 60.4323508], [22.2574307, 60.432353], [22.2562632, 60.4323511], [22.255197, 60.4322687], [22.2549063, 60.4322403], [22.2546108, 60.4322031], [22.2542454, 60.4321497], [22.2535673, 60.4320551], [22.2520182, 60.431839], [22.2516319, 60.4317851], [22.251479, 60.4317639], [22.2505003, 60.4316459], [22.2502079, 60.4316075], [22.2496976, 60.4315406], [22.248865, 60.4314346], [22.2486055, 60.4314016], [22.2464404, 60.4310864], [22.246

In [535]:
graph.roads[42684571]

[21646150, 1101337726, 243674925, 533774011, 32919954, 30390838, 21646152]

In [514]:
graph.node_path_geojson([21646150, 1101337726, 243674925, 533774011, 32919954, 30390838, 21646152])

{"coordinates": [[22.2458997, 60.4310094], [22.2456772, 60.430982], [22.2449676, 60.4308949], [22.2436039, 60.4307058], [22.2420127, 60.4304877], [22.2408519, 60.4303133], [22.2403586, 60.4302491]], "type": "LineString"}

In [ ]:
neighbour_nodes = set() #Only collect unique neighbours
way_list = test.node2way(338507468)
for way_id in way_list:
    way = test.ways[way_id]
    way_index = way.index(338507468)
    forward_node = way_index + 1
    backward_node = way_index - 1
    try:
        forward = way[forward_node]
        neighbour_nodes.add(forward)
    except:
        pass
    try:
        if backward_node > 0:
            backward = way[backward_node]
            neighbour_nodes.add(backward)
    except:
        pass

In [ ]:
neighbour_nodes

In [ ]:
temp = [1,2,3]
temp[3]

In [ ]:
test.ways

In [ ]:
m

In [ ]:
deque([2,4]).sort()

In [ ]:
graph = Aplanner(0.738144,51.809888,0.750504,51.822623)

#Define Problem
start = 308546906
goal = 316303594

search_path = []
explored_nodes = set()


def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
            
        current = frontier.get()
        
        #Check if the goal has been expanded
        if goal == came_from[current]:
            print("Solution Found")
            break
        
        children = graph.neigbour_nodes(current)
        
        for child in children:
            
            child_cost = cost_so_far[current] + graph.node_distance(current, child) - graph.node_distance(child, goal)
            
            if child not in came_from or child_cost < cost_so_far[child]:
                
                priority = child_cost
                frontier.put(child, priority)
                came_from[child] = current
                cost_so_far[child] = child_cost

                
    return came_from

search_results = a_star_search(graph, start, goal)
path = path_construction(start, goal, search_results)

In [ ]:
for node in test.turns:
    print(test.neigbour_nodes(node))

In [ ]:
test.ways[25643156]

In [ ]:
node_ways = {}
node_list = []

for way in list(test.ways.keys()):
    for node in test.ways[way]:
        if node not in list(node_ways.keys()):
            node_ways[node] = [way]
            node_list.append(node)
        else:
            print("Node already in dict")
            list_ = node_ways[node]
            list_.append(way)
            node_ways[node] = list_
#node_ways

In [ ]:
test.ways[32702976]

In [ ]:
print(len(set(node_list)))
print(len(node_list))

In [ ]:
test.turns

In [ ]:
test.turns

In [ ]:
if 1583202304 in test.ways.keys():
    print('yes')

In [ ]:
for key in node_ways.keys():
    if len(node_ways[key]) > 1:
        print (len(node_ways[key]))

In [ ]:
test.node_ways[1583202304]

In [ ]:
def node2loc(node_id):
    for node in node_data:
        if node_id == node['data']['id']:
            return (node['data']['lon'], node['data']['lat'])

In [ ]:
node_data[0]

In [ ]:
for dict_ in map_nodes:
    if dict_['type'] == 'way':
        way = dict_
        if 'highway' in way['data']['tag'].keys():
            print(way, '\n')

In [ ]:
set_ = set()
for dict_ in map_nodes:
    set_.add(tuple(dict_['type']))
    